<center><strong><font size=+3>Calibration demonstration</font></center>
<br><br>
</center>
<center><strong><font size=+2>Matyas Molnar</font><br></strong></center>
<br><center><strong><font size=+1>Astrophysics Group, Cavendish Laboratory, University of Cambridge</font></strong></center>

Gridded plots of redundant visibilities at each stage of the calibration pipeline, inspired by [Josh Dillon's plotting notebook](https://github.com/HERA-Team/hera_sandbox/blob/master/jsd/Data_Inspection/Abscal_Omnical_Demonstration_Plots.ipynb).

In [ ]:
import glob
import os
from copy import deepcopy

import numpy as np
from matplotlib import pyplot as plt

from hera_cal.apply_cal import calibrate_in_place
from hera_cal.io import HERAData, HERACal
from hera_cal.redcal import get_reds

from cal_utils import apply_mask, ex_badants, plot_reds

In [ ]:
%matplotlib inline

In [ ]:
JD = 2458098.43869
pol = 'ee'
bl = (38, 39, 'ee') # baseline examined in this notebook

In [ ]:
plot_figs = False
if plot_figs:
    import matplotlib as mpl
    mpl.rcParams['figure.dpi'] = 300
    
from matplotlib import rc
rc('font',**{'family':'serif','serif':['cm']})
rc('text', usetex=True)
rc('text.latex', preamble=r'\usepackage{amssymb} \usepackage{amsmath}')

## Reading the data

In [ ]:
analysis_folder = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/{}'.format(int(JD))
raw_data_fn = 'zen.{}.HH.uvh5'.format(JD)

hera_pkgs = '/users/heramgr/hera_software/'
if not os.path.exists(analysis_folder): # working locally
    analysis_folder = '/Users/matyasmolnar/Downloads/HERA_Data/sample_data'
    hera_pkgs = '/Users/matyasmolnar/Downloads/HERA_Data/hera_packages/'

In [ ]:
raw_data_file = os.path.join(analysis_folder, raw_data_fn)
hd = HERAData(raw_data_file)

In [ ]:
reds = get_reds(hd.antpos, pols=[pol])
red_dict = {bl: bl_group[0] for bl_group in reds for bl in bl_group}
bl_red = red_dict[bl]
print('Baseline {} corresponds to redundant baseline group of type {}'.format(bl, bl_red))

In [ ]:
red_idx = np.where((np.asarray([redg[0] for redg in reds]) == np.asarray(bl_red)).all(axis=1))[0][0]
slct_redg = reds[red_idx][:15] # only choose first 15..
print('Baselines in redundant group {}: {}'.format(bl_red, slct_redg))

## Raw visibilities

In [ ]:
# slct_redg = [(12, 13, 'ee'), (25, 26, 'ee'), (36, 37, 'ee')]

In [ ]:
red_data, red_flags, nsamples = hd.read(bls=slct_redg)  # only load redundant group

In [ ]:
plot_reds(red_data, slct_redg, \
          'Phase of Raw Visibilities for Baselines Redundant to {}'.format(bl_red), savefig=None)

## Bad antennas

In [ ]:
bad_ants_file = os.path.join(hera_pkgs, 'hera_pipelines/pipelines/h1c/idr2/v2/bad_ants/{}.txt'.format(int(JD)))
bad_ants = np.loadtxt(bad_ants_file).astype(int)
print('Bad antennas for JD {} are: {}'.format(int(JD), list(bad_ants)))

In [ ]:
bad_bls = []
for (i, j, pol) in slct_redg:
    if i in bad_ants or j in bad_ants:
        bad_bls.append((i, j, pol))
print('Bad baselines are {}'.format(bad_bls))

In [ ]:
red_data_ex_ant = deepcopy(red_data)
red_data_ex_ant = ex_badants(red_data_ex_ant, bad_ants)

In [ ]:
plot_reds(red_data_ex_ant, slct_redg, \
          'Phase of Raw Visibilities for Baselines Redundant to {}'.format(bl_red), \
          bad_bls=bad_bls, savefig=None)

## Firstcal

In [ ]:
# load firstcal gains and flags
firstcal_fn = raw_data_fn.replace('uvh5', 'first.calfits')
firstcal_file = os.path.join(analysis_folder, firstcal_fn)
hc = HERACal(firstcal_file)
fc_gains, fc_flags, _, _ = hc.read()

In [ ]:
fc_red_data = deepcopy(red_data)
fc_red_flags = deepcopy(red_flags)
calibrate_in_place(fc_red_data, fc_gains, data_flags=fc_red_flags, cal_flags=fc_flags)
fc_red_data = ex_badants(fc_red_data, bad_ants)

In [ ]:
plot_reds(fc_red_data, slct_redg, \
          'Phase of Firstcal Visibilities for Baselines Redundant to {}'.format(bl_red), \
          flags=fc_red_flags, bad_bls=bad_bls)

## Redcal

In [ ]:
# load omnical gains and flags
omnical_fn = raw_data_fn.replace('uvh5', 'omni.calfits')
omnical_file = os.path.join(analysis_folder, omnical_fn)
hc = HERACal(omnical_file)
oc_gains, oc_flags, _, _ = hc.read()

In [ ]:
oc_red_data = deepcopy(red_data)
oc_red_flags = deepcopy(red_flags)
calibrate_in_place(oc_red_data, oc_gains, data_flags=oc_red_flags, cal_flags=oc_flags)
oc_red_data = ex_badants(oc_red_data, bad_ants)

In [ ]:
plot_reds(oc_red_data, slct_redg, \
          'Phase of Redcal Visibilities for Baselines Redundant to {}'.format(bl_red), \
          flags=oc_red_flags, bad_bls=bad_bls, savefig=None)

 ## Abscal

In [ ]:
# load abscal gains and flags
abscal_fn = raw_data_fn.replace('uvh5', 'abs.calfits')
abscal_fn_file = os.path.join(analysis_folder, abscal_fn)
hc = HERACal(abscal_fn_file)
abs_gains, abs_flags, _, _ = hc.read()

In [ ]:
abs_red_data = deepcopy(red_data)
abs_red_flags = deepcopy(red_flags)
calibrate_in_place(abs_red_data, abs_gains, data_flags=abs_red_flags, cal_flags=abs_flags)
abs_red_data = ex_badants(abs_red_data, bad_ants)

In [ ]:
plot_reds(abs_red_data, slct_redg, \
          'Phase of Abscal Visibilities for Baselines Redundant to {}'.format(bl_red), \
          flags=abs_red_flags, bad_bls=bad_bls, savefig=None)

## Abscal + masked band edges

In [ ]:
abs_red_data_masked = deepcopy(abs_red_data)
abs_red_data_masked = apply_mask(abs_red_data_masked, flags=None, edge_cut=100)

In [ ]:
plot_reds(abs_red_data_masked, slct_redg, \
          'Phase of Abscal Visibilities for Baselines Redundant to {}'.format(bl_red), \
          bad_bls=bad_bls)

## Flagged abscal

In [ ]:
# load flagged abscal gains and flags
flg_abscal_fn = raw_data_fn.replace('uvh5', 'flagged_abs.calfits')
flg_abscal_fn_file = os.path.join(analysis_folder, flg_abscal_fn)
hc = HERACal(flg_abscal_fn_file)
flg_abs_gains, flg_abs_flags, _, _ = hc.read()

In [ ]:
flg_abs_red_data = deepcopy(red_data)
flg_abs_red_flags = deepcopy(red_flags)
calibrate_in_place(flg_abs_red_data, flg_abs_gains, data_flags=flg_abs_red_flags, cal_flags=flg_abs_flags)
flg_abs_red_data = ex_badants(flg_abs_red_data, bad_ants)
flg_abs_red_data = apply_mask(flg_abs_red_data, flags=None, edge_cut=100)

In [ ]:
plot_reds(flg_abs_red_data, slct_redg, \
          'Phase of Abscal Visibilities for Baselines Redundant to {}'.format(bl_red), \
          flags=flg_abs_red_flags, bad_bls=bad_bls)

## Smoothed abs

In [ ]:
# load flagged abscal gains and flags
smooth_abscal_fn = raw_data_fn.replace('uvh5', 'smooth_abs.calfits')
smooth_abscal_fn_file = os.path.join(analysis_folder, smooth_abscal_fn)
hc = HERACal(smooth_abscal_fn_file)
smooth_abs_gains, smooth_abs_flags, _, _ = hc.read()

In [ ]:
smooth_abs_red_data = deepcopy(red_data)
smooth_abs_red_flags = deepcopy(red_flags)
calibrate_in_place(smooth_abs_red_data, smooth_abs_gains, data_flags=smooth_abs_red_flags, \
                   cal_flags=smooth_abs_flags)
smooth_abs_red_data = ex_badants(smooth_abs_red_data, bad_ants)
smooth_abs_red_data = apply_mask(smooth_abs_red_data, flags=smooth_abs_red_flags, \
                                 edge_cut=100)

In [ ]:
plot_reds(smooth_abs_red_data, slct_redg, \
          'Phase of Abscal Visibilities for Baselines Redundant to {}'.format(bl_red), \
          flags=smooth_abs_red_flags, bad_bls=bad_bls, savefig=None)

## LST-Binned

In [ ]:
hd_lstb1 = HERAData('/Users/matyasmolnar/Downloads/HERA_Data/sample_data/zen.grp1.of1.LST.1.31552.HH.OCRSL.uvh5')
hd_lstb2 = HERAData('/Users/matyasmolnar/Downloads/HERA_Data/sample_data/zen.grp1.of1.LST.1.40949.HH.OCRSL.uvh5')

In [ ]:
lstb_data1, lstb_flags1, nsamples1 = hd_lstb1.read(bls=slct_redg)
lstb_data2, lstb_flags2, nsamples2 = hd_lstb2.read(bls=slct_redg)

In [ ]:
lstb_data = {k: np.concatenate((v[43:], lstb_data2[k][:13]), axis=0)  for k, v in lstb_data1.items()}
lstb_flags = {k: np.concatenate((v[43:], lstb_flags2[k][:13]), axis=0)  for k, v in lstb_flags1.items()}

In [ ]:
plot_reds(lstb_data, slct_redg, \
          'Visibility Phase'.format(bl_red), \
          flags=lstb_flags, bad_bls=bad_bls, savefig=None)